#### Algorithmics for Data Mining Project 2
# Regression
#### Ondrej Herman, Darryl Abraham

#### Importing Libraries

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

#### Reading Data

In [2]:
df = pd.read_csv('data/preprocessed_data.csv')

In [3]:
df.head()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode,album_name,artists,explicit,track_id,time_signature,track_name,key,track_genre
0,0.975633,-0.730859,-0.845908,-1.889980,-1.784744,-0.078993,1.831732,-0.504094,-0.591211,-0.798690,-1.489717,1,Ghost (Acoustic),Ben Woodward,False,4qPNDBW1i3p13qLCt0Ki3A,4,Ghost - Acoustic,1,acoustic
1,1.065299,-0.160332,-0.742186,-1.122669,-0.293288,-0.273826,-0.315499,-0.504112,-0.507167,-1.365688,-1.528312,1,To Begin Again,Ingrid Michaelson;ZAYN,False,1iJBSr7s7jYXzM8EGcbK5b,4,To Begin Again,0,acoustic
2,1.692961,-0.243214,-1.733304,-2.312994,-2.039252,-0.457309,1.774593,-0.503883,-0.428376,-1.276974,1.987859,1,Crazy Rich Asians (Original Motion Picture Sou...,Kina Grannis,False,6lfxq3CG4xtTiEg7opyCyx,3,Can't Help Falling In Love,0,acoustic
3,2.186125,-0.271919,0.295030,-0.788711,-0.282750,-0.303145,0.463399,-0.504112,-0.686285,-1.184403,-0.073348,1,Hold On,Chord Overstreet,False,5vjLSffimiIP26QG5WcN2K,4,Hold On,2,acoustic
4,1.827460,0.012776,-0.920818,-1.965518,-0.111952,-0.464875,1.630242,-0.504103,-0.642162,-1.533474,0.638339,1,Is There Anybody Out There?,A Great Big World;Christina Aguilera,False,6Vc5wAMmXdKIAM7WUoEb7N,3,Say Something,2,acoustic


In [60]:
df.describe()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,spanish,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
count,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,...,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000,99510.000000
mean,0.005015,-0.062121,0.011782,0.013758,0.061775,-0.222461,-0.009693,-0.479500,-0.140225,0.008574,...,0.009054,0.009135,0.008914,0.008602,0.008411,0.008401,0.008974,0.008904,0.008622,0.008431
std,0.996233,0.684194,0.987992,0.996195,0.865703,0.354958,0.998565,0.061442,0.718848,0.995597,...,0.094723,0.095139,0.093991,0.092348,0.091327,0.091273,0.094306,0.093939,0.092455,0.091435
min,-1.490184,-1.977955,-3.266080,-2.549866,-3.335651,-0.800630,-0.947037,-0.504112,-1.073149,-1.828544,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.728023,-0.501955,-0.615415,-0.645585,-0.297663,-0.464875,-0.897717,-0.504112,-0.610121,-0.814118,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.078972,-0.149148,0.087587,0.193288,0.275576,-0.352326,-0.456843,-0.504054,-0.444134,-0.031120,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.751467,0.274843,0.732966,0.849278,0.660321,-0.108313,0.830293,-0.495454,0.180942,0.809735,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.993119,2.753555,2.404037,1.425755,2.004241,1.374683,2.048260,0.103215,3.069948,2.009303,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Data Preprocessing

In [4]:
# RUN ONCE
df = df.drop(columns=['track_name', 'album_name', 'artists', 'track_id'])
df['explicit'] = df['explicit'].astype(int)
dummy = pd.get_dummies(df['track_genre']).astype(int)
df = pd.concat([df.drop(columns='track_genre'), dummy], axis=1)
df.head()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,...,spanish,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
0,0.975633,-0.730859,-0.845908,-1.889980,-1.784744,-0.078993,1.831732,-0.504094,-0.591211,-0.798690,...,0,0,0,0,0,0,0,0,0,0
1,1.065299,-0.160332,-0.742186,-1.122669,-0.293288,-0.273826,-0.315499,-0.504112,-0.507167,-1.365688,...,0,0,0,0,0,0,0,0,0,0
2,1.692961,-0.243214,-1.733304,-2.312994,-2.039252,-0.457309,1.774593,-0.503883,-0.428376,-1.276974,...,0,0,0,0,0,0,0,0,0,0
3,2.186125,-0.271919,0.295030,-0.788711,-0.282750,-0.303145,0.463399,-0.504112,-0.686285,-1.184403,...,0,0,0,0,0,0,0,0,0,0
4,1.827460,0.012776,-0.920818,-1.965518,-0.111952,-0.464875,1.630242,-0.504103,-0.642162,-1.533474,...,0,0,0,0,0,0,0,0,0,0


#### Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='popularity'), df['popularity'], test_size=0.7, random_state=42)

#### Evaluation framework

In [56]:
def cross_validate(model, X, y, num_folds):
    # Initialize KFold cross-validation object
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    # Initialize lists to store metrics and coefficients
    mse_values = []
    r2_values = []
    intercepts = []
    coefficients = []
    
    # Perform cross-validation
    for train_index, test_index in kf.split(X, y):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Fit the model on the training data
        model.fit(X_train, y_train)
        
        # Predict the test data
        y_pred = model.predict(X_test)
        
        # MSE
        mse_values.append(mean_squared_error(y_test, y_pred))
        # R2
        r2_values.append(r2_score(y_test, y_pred))
        # Get intercept
        intercepts.append(model.intercept_)
        # Get the coefficients
        coefficients.append(model.coef_)
        
    # Calculate the mean and variance of the metrics
    mean_mse = np.mean(mse_values)
    variance_mse = np.var(mse_values)
    print(f'Mean MSE: {mean_mse}, Variance MSE: {variance_mse}')
    
    mean_r2 = np.mean(r2_values)
    variance_r2 = np.var(r2_values)
    print(f'Mean R2: {mean_r2}, Variance R2: {variance_r2}')
    
    mean_intercept = np.mean(intercepts)
    variance_intercept = np.var(intercepts)
    
    transposed_data = np.array(coefficients).T
    means = np.mean(transposed_data, axis=1)
    variances = np.var(transposed_data, axis=1)
    coefficients_df = pd.DataFrame({'Mean Coefficient': means, 'Variance Coefficient': variances}, index=X.columns)
    
    eval_dict = {
        'mse': (mean_mse, variance_mse, mse_values),
        'r2': (mean_r2, variance_r2, r2_values),
        'intercept': (mean_intercept, variance_intercept, intercepts),
        'Coefficients': coefficients_df
    }
    
    return eval_dict

## Linear Regression
#### Model Training

In [7]:
model = LinearRegression()
num_folds = 10

X = df.drop(columns='popularity')
y = df['popularity']

lm = cross_validate(model, X, y, num_folds)

Mean MSE: 0.6972646524094266, Variance MSE: 0.00024888149290919393
Mean R2: 0.2974040041522366, Variance R2: 0.00011564842058362452


#### Model Evaluation

In [8]:
print(f'Mean Intercept: {lm['intercept'][0]}, Variance Intercept: {lm['intercept'][1]}')
lm['Coefficients']

Mean Intercept: -0.22393744515459338, Variance Intercept: 0.00014805205007365534


,Mean Coefficient,Variance Coefficient
duration_ms,0.019984,0.000005
danceability,0.051489,0.000002
energy,-0.051250,0.000002
loudness,0.046014,0.000004
speechiness,-0.083718,0.000005
...,...,...
techno,0.204282,0.000089
trance,0.148437,0.000062
trip-hop,0.048567,0.000089
turkish,0.296828,0.000005


## Ridge Regression
#### Model Training
Hyperparameter tuning

In [40]:
alphas = np.logspace(-3, 3, 7).tolist()
alphas.extend([round(x, 1) for x in np.arange(2, 2.5, 0.1)]) # TUNED RANGE
best_alpha_ridge = None
best_mse_ridge = 999999
for alpha in alphas:
    model = Ridge(alpha=alpha)
    print(f'Alpha: {alpha}')
    rm = cross_validate(model, X_train, y_train, num_folds)
    if best_alpha_ridge is None or rm['mse'][0] < best_mse_ridge:
        best_alpha_ridge = alpha
        best_mse_ridge = rm['mse'][0]

Alpha: 0.001
Mean MSE: 0.7003080986293716, Variance MSE: 0.00010361929027292957
Mean R2: 0.2935125367328365, Variance R2: 0.00014810039075017112
Alpha: 0.01
Mean MSE: 0.7003078877795919, Variance MSE: 0.00010361838366065247
Mean R2: 0.29351275529956017, Variance R2: 0.0001480913086312103
Alpha: 0.1
Mean MSE: 0.7003058278256564, Variance MSE: 0.00010360933411883299
Mean R2: 0.2935148919614686, Variance R2: 0.00014800055613188476
Alpha: 1.0
Mean MSE: 0.7002900367829281, Variance MSE: 0.00010352048863713694
Mean R2: 0.2935314043716912, Variance R2: 0.00014709983937459358
Alpha: 10.0
Mean MSE: 0.7005706753182468, Variance MSE: 0.00010278822994540403
Mean R2: 0.2932538074371879, Variance R2: 0.00013871821336841592
Alpha: 100.0
Mean MSE: 0.723878088563794, Variance MSE: 0.00010420138910166837
Mean R2: 0.2697778931381495, Variance R2: 8.915844613288033e-05
Alpha: 1000.0
Mean MSE: 0.8772422968422985, Variance MSE: 0.00015636822549189717
Mean R2: 0.11513327312752045, Variance R2: 2.107899798731

#### Model Evaluation

In [42]:
print(f'Best Alpha: {best_alpha_ridge}')

Best Alpha: 2.2


In [43]:
rm = cross_validate(Ridge(alpha=best_alpha_ridge), X, y, num_folds)

Mean MSE: 0.6972616348120452, Variance MSE: 0.00024820551315640836
Mean R2: 0.29740693368911564, Variance R2: 0.0001151153868271697


In [44]:
print(f'Mean Intercept: {rm['intercept'][0]}, Variance Intercept: {rm['intercept'][1]}')
rm['Coefficients']

Mean Intercept: -0.22552025216644783, Variance Intercept: 0.00014647048067075827


,Mean Coefficient,Variance Coefficient
duration_ms,0.019969,0.000004
danceability,0.051512,0.000002
energy,-0.051385,0.000002
loudness,0.046415,0.000004
speechiness,-0.084196,0.000005
...,...,...
techno,0.203874,0.000088
trance,0.148039,0.000061
trip-hop,0.048550,0.000088
turkish,0.295980,0.000005


## Lasso Regression
#### Model Training
Hyperparameter tuning

In [51]:
alphas = np.logspace(-3, 3, 7).tolist()
#alphas.extend([round(x, 1) for x in np.arange(0.0005, 0.001, 0.00001)]) # tuned range
best_alpha_lasso = None
best_mse_lasso = 999999
for alpha in alphas:
    model = Lasso(alpha=alpha)
    print(f'Alpha: {alpha}')
    lso = cross_validate(model, X_train, y_train, num_folds)
    if best_alpha_lasso is None or lso['mse'][0] < best_mse_lasso:
        best_alpha_lasso = alpha
        best_mse_lasso = lso['mse'][0]

Alpha: 0.001
Mean MSE: 0.7110773325201869, Variance MSE: 0.00010259119963038146
Mean R2: 0.28267690380175886, Variance R2: 0.00010752150741696713
Alpha: 0.01
Mean MSE: 0.9582693332383194, Variance MSE: 0.00021138652175844196
Mean R2: 0.03342351377048712, Variance R2: 7.520966719030297e-06
Alpha: 0.1
Mean MSE: 0.991783846067043, Variance MSE: 0.00024171747697925738
Mean R2: -0.0003699560500710186, Variance R2: 1.805173388924453e-07
Alpha: 1.0
Mean MSE: 0.991783846067043, Variance MSE: 0.00024171747697925738
Mean R2: -0.0003699560500710186, Variance R2: 1.805173388924453e-07
Alpha: 10.0
Mean MSE: 0.991783846067043, Variance MSE: 0.00024171747697925738
Mean R2: -0.0003699560500710186, Variance R2: 1.805173388924453e-07
Alpha: 100.0
Mean MSE: 0.991783846067043, Variance MSE: 0.00024171747697925738
Mean R2: -0.0003699560500710186, Variance R2: 1.805173388924453e-07
Alpha: 1000.0
Mean MSE: 0.991783846067043, Variance MSE: 0.00024171747697925738
Mean R2: -0.0003699560500710186, Variance R2: 1

#### Model Evaluation

In [52]:
print(f'Best Alpha: {best_alpha_lasso}')

Best Alpha: 0.001


In [53]:
lso = cross_validate(Lasso(alpha=best_alpha_lasso), X_test, y_test, num_folds)

Mean MSE: 0.7081833065639392, Variance MSE: 0.0001421505037833921
Mean R2: 0.2865395354461732, Variance R2: 5.186740757888127e-05


In [54]:
print(f'Mean Intercept: {lso['intercept'][0]}, Variance Intercept: {lso['intercept'][1]}')
lso['Coefficients']

Mean Intercept: -0.1191832712495208, Variance Intercept: 9.609776088691024e-05


,Mean Coefficient,Variance Coefficient
duration_ms,0.021308,0.000002
danceability,0.047779,0.000001
energy,-0.061137,0.000006
loudness,0.066483,0.000002
speechiness,-0.108769,0.000012
...,...,...
techno,0.011560,0.000053
trance,0.000000,0.000000
trip-hop,0.000000,0.000000
turkish,0.104920,0.000015


## Generalised Linear Models
#### Model Training

In [32]:
from sklearn.linear_model import TweedieRegressor

In [33]:
model = TweedieRegressor(power=0, alpha=1, link='identity', solver='lbfgs', max_iter=1000)
glm = cross_validate(model, X_train, y_train, num_folds)

Mean MSE: 0.9791664575010197, Variance MSE: 0.00022458821132053422
Mean R2: 0.012350825750298545, Variance R2: 8.490654003698215e-07


Horrible, not worth it